# Práctica 2: GPU Programming (CUDA)

## Preparación del entorno

Eliminación de datos innecesarios creados por Google Collab

In [1]:
!rm -rf sample_data

Descarga de `Numba` en caso de no encontrarse en el sistema

In [ ]:
!pip install numba --upgrade

Paquetes necesarios para el correcto funcionamiento del código

In [1]:
import numpy as np
from numba import cuda, float32
import time
import math

## 2.1 Compulsory assignment #1: Matrix transpose

In [ ]:
Ax = 5
Ay = 7
Bx = Ay
By = Ax

def create_matrix_1(Ax: int = 5000, Ay: int = 7000) -> tuple:
  return np.random.rand(Ax, Ay), np.zeros((Ay, Ax))

In [ ]:
############################# Explicaciones pal iñigo #############################

# Esto es un decorador, sirve para que la máquina sepa como
# ejecutar el código siguiente, en este caso, sirve para indicarle
# que la función se ejecuta en gpu (tarjeta gráfica) (paralelizando)
# en vez de en cpu (procesador) (que normalmente es en secuencial)
@cuda.jit
def transpose_parallel(A, B):
    # Esta función sirve para obtener la posición del hilo actual
    # en una matriz de 2 dimensiones
    i, j = cuda.grid(2)

    # Como verás en una explicación después, algunas veces por conveniencia
    # se crean hilos de más, haciendo que haya más hilos que posiciones en
    # esto puede causar problemas en el acceso a las posiciones, asi que tenemos
    # que asegurarnos que el hilo que estamos usando esta dentro de la posición
    # de la matriz
    if i < B.shape[0] and j < B.shape[1]:
        B[i, j] = A[j, i]

def transpose_sequential(A, B):
    for i in range(0, B.shape[0]):
        for j in range(0, B.shape[1]):
            B[i, j] = A[j, i]

In [ ]:
A, B_seq = create_matrix_1()

t_start = time.time()
transpose_sequential(A, B_seq)
t_finish = time.time()

t_cpu = t_finish - t_start

print(f"A = {A}")
print(f"B sequential = {B_seq}")
print(f"Tiempo ejecución cpu = {t_cpu}")

A = [[0.5332116  0.30702299 0.10948742 ... 0.38825487 0.48623714 0.39570486]
 [0.87560167 0.82235289 0.05881936 ... 0.28517249 0.229938   0.54345414]
 [0.17976587 0.6223765  0.46298395 ... 0.76090316 0.98335124 0.16403582]
 ...
 [0.6870055  0.42397955 0.1101008  ... 0.70033198 0.324623   0.03178206]
 [0.74661687 0.26387922 0.1424545  ... 0.03242135 0.19320822 0.19844304]
 [0.72395246 0.39598295 0.36207431 ... 0.00946169 0.04915248 0.33458658]]
B sequential = [[0.5332116  0.87560167 0.17976587 ... 0.6870055  0.74661687 0.72395246]
 [0.30702299 0.82235289 0.6223765  ... 0.42397955 0.26387922 0.39598295]
 [0.10948742 0.05881936 0.46298395 ... 0.1101008  0.1424545  0.36207431]
 ...
 [0.38825487 0.28517249 0.76090316 ... 0.70033198 0.03242135 0.00946169]
 [0.48623714 0.229938   0.98335124 ... 0.324623   0.19320822 0.04915248]
 [0.39570486 0.54345414 0.16403582 ... 0.03178206 0.19844304 0.33458658]]
Tiempo ejecución cpu = 14.932383298873901


In [ ]:
############################# Explicaciones pal iñigo #############################

_, B_par = create_matrix_1()

# Esto sirve para meter el la vram de la gpu las matrices
A_device = cuda.to_device(A)
B_device = cuda.to_device(B_par)

# Para el uso optimo de la gpu, tenemos que tener en cuenta que
# para maxima eficiencia de un warp (conjunto de hilos) este tiene
# que tener 32, como estamos tratando datos en 2D, dividimos esos
# 32 hilos en dos
threads_per_block = (16, 16)
# Para determinar el número de bloques que debemos usar simplemente
# dividimos el eje entre el número de hilos del eje aplicando la función
# techo para que no se queden posiciones libres sin hilos asignados
# (aunque para ello debamos crear hilos que no se vayan a usar)
blocks_X = math.ceil(B_par.shape[0] / threads_per_block[0])
blocks_Y = math.ceil(B_par.shape[1] / threads_per_block[1])
blocks_total = (blocks_X, blocks_Y)

t_start = time.time()
# Llamamos a la función de la transpuesta en paralelo, para ello, hay
# que indicar el número de bloques que se van a usar en cada dimensión,
# junto al número de hilos que va a tener cada bloque
transpose_parallel[blocks_total, threads_per_block](A_device, B_device)

# Esto es MUY importante poque como lo que acabamos de hacer es "una
# pelea salvaje en la jungla" por los calculos de los hilos, debemos
# asegurarnos que todos acaban, para eso sirve esta función, en caso de
# que un hilo falte por realizar sus calculos no se continuará. Esto en
# informática se llama "Condición de carrera" y es muy importante para
# que funcione bien un codigo
cuda.synchronize()
t_finish = time.time()

# Tras realizar todos los calculos necesarios, se copia de la vram de la gpu
# a la ram de la cpu
B_par = B_device.copy_to_host()

t_gpu = t_finish - t_start

print(f"A = {A}")
print(f"B parallel = {B_par}")
print(f"Tiempo ejecución gpu = {t_gpu}")

A = [[0.5332116  0.30702299 0.10948742 ... 0.38825487 0.48623714 0.39570486]
 [0.87560167 0.82235289 0.05881936 ... 0.28517249 0.229938   0.54345414]
 [0.17976587 0.6223765  0.46298395 ... 0.76090316 0.98335124 0.16403582]
 ...
 [0.6870055  0.42397955 0.1101008  ... 0.70033198 0.324623   0.03178206]
 [0.74661687 0.26387922 0.1424545  ... 0.03242135 0.19320822 0.19844304]
 [0.72395246 0.39598295 0.36207431 ... 0.00946169 0.04915248 0.33458658]]
B parallel = [[0.5332116  0.87560167 0.17976587 ... 0.6870055  0.74661687 0.72395246]
 [0.30702299 0.82235289 0.6223765  ... 0.42397955 0.26387922 0.39598295]
 [0.10948742 0.05881936 0.46298395 ... 0.1101008  0.1424545  0.36207431]
 ...
 [0.38825487 0.28517249 0.76090316 ... 0.70033198 0.03242135 0.00946169]
 [0.48623714 0.229938   0.98335124 ... 0.324623   0.19320822 0.04915248]
 [0.39570486 0.54345414 0.16403582 ... 0.03178206 0.19844304 0.33458658]]
Tiempo ejecución gpu = 0.3652820587158203


`Pal iñigo: Como puedes darte cuenta, el tiempo de ejecución de la gpu es más alto que el de la cpu cuando por lógica debería ser al revés al hacerlo en paralelo, esto se debe a que al ser un tamaño de matrix pequeño, es mayor el peso de creación de hilos y ponerlos en ejecución que el de la propia ejecución de la traspuesta. Si se decidiese poner un tamaño mucho más grande de matriz se podría ver como esto se cumple.`

In [ ]:
speedup = t_cpu / t_gpu

print(f"Speedup = {speedup}")

Speedup = 40.879049333465616


## 2.2 Compulsory assignment #2: Average Rows/Cols I

In [4]:
def avg_Cols_sequential(input, output):
    for y in range(input.shape[1]):
        output[y] = 0.0
        for x in range(input.shape[0]):
            output[y] += input[x, y]
        output[y] /= input.shape[0]

def avg_Rows_sequential(input, output):
    #TODO (esto es un comentario especial de informáticos
    # sirve para indicar que queda por hacer cosas (del verbo en ingles
    # to do), siempre no sepas que hacer busca un TODO por el código)
    pass

############################# Explicaciones pal iñigo #############################

@cuda.jit
def avg_Cols_parallel(input, output):
    # En este ejercicio no necesita que hagamos un recorrido en matriz,
    # sino en filas, por lo tanto, solo usaremos un eje
    y = cuda.grid(1)
    
    if y < input.shape[1]:
        sum_val = 0.0
        # Cada hilo suma todos los valores de la columna que le ha tocado
        for x in range(input.shape[0]):
            sum_val += input[x, y]
        # Y luego divide por el número de elementos en la columna
        output[y] = sum_val / input.shape[0]

@cuda.jit
def avg_Rows_parallel(input, output):
    #TODO
    pass

In [5]:
np.random.seed(0)

Ax = 5
Ay = 7

A = np.random.rand(Ax, Ay)
B = np.zeros(Ay)

#TODO: calculo de tiempo en cpu

avg_Cols_sequential(A, B)

print("input\n", A)
print("output\n", B)

input
 [[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
  0.43758721]
 [0.891773   0.96366276 0.38344152 0.79172504 0.52889492 0.56804456
  0.92559664]
 [0.07103606 0.0871293  0.0202184  0.83261985 0.77815675 0.87001215
  0.97861834]
 [0.79915856 0.46147936 0.78052918 0.11827443 0.63992102 0.14335329
  0.94466892]
 [0.52184832 0.41466194 0.26455561 0.77423369 0.45615033 0.56843395
  0.0187898 ]]
output
 [0.56652589 0.52842455 0.41030162 0.61234724 0.56535556 0.55914761
 0.66105218]


In [ ]:
# Pal iñigo: haz tu la ejecución del codigo en paralelo según los comentarios que te he puesto en el ejercicio anterior
# IMPORTANTE: el codigo que da el profesor esta AL REVÉS, en la funcion de media de columnas hace de filas y viceversa

#A no hace falta ponerlo porque ya esta inicializado de antes
B = np.zeros(Ay)

#TODO: ejecución del calculo en paralelo

#TODO: calculo del tiempo en gpu

In [ ]:
#TODO: calculo del speedup

## 2.3 Compulsory assignment #3: Average Rows/Cols II (2 points)

In [ ]:
# Pal iñigo: este tambien lo haces tu que es la segunda parte del anterior
# sin embargo, ten en cuenta que se usa MEMORIA COMPARTIDA, puedes mirar el siguiente
# ejercicio para saber como usarlo

#TODO

## 2.4 Optional assignment #1: Mean_3x3 (2 points)

In [46]:
def Mean_3x3_sequential(input, output):
    for x in range(input.shape[0]):
        for y in range(input.shape[1]):
            output[x, y] = 0.0
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if x + i >= 0 and x + i < input.shape[0] and \
                        y + j >= 0 and y + j < input.shape[1]:
                        output[x, y] += input[x + i, y + j]
            output[x, y] /= 9.0

################################ Explicaciones pal iñigo ################################
@cuda.jit
def Mean_3x3_parallel(input, output, aux):

    # Recuerda que cada bloque se encarga de una matriz más pequeña que la matriz inicial
    # que nosotros hemos definido de 16x16, en este ejercicio en concreto nos piden
    # calcular la media de los números adyacentes a una posición y el valor de la posición
    # también (y la media siempre se hace dividiendo entre 9), para hacer el ejercicio más
    # sencillo para nosotros y los hilos, he optado por hacer más grande la matriz de memoria
    # compartida, haciendo que la matriz sea 2 veces más ancha y larga.
    # Ej:
    #                                           [[0.         0.         0.         0.         0.        ]
    # [[0.98469016 0.22750222 0.99213766]       [0.         0.98469019 0.22750223 0.99213767 0.        ]
    # [0.25682925 0.49983334 0.25695094]    ->  [0.         0.25682923 0.49983335 0.25695094 0.        ]
    # [0.8441824  0.22751885 0.92430338]]       [0.         0.84418243 0.22751886 0.92430335 0.        ]
    #                                           [0.         0.         0.         0.         0.        ]]
    shared_mem = cuda.shared.array((16 + 2, 16 + 2), dtype=float32)

    # Obtenemos el número total de filas y columnas de la matriz de entrada
    rows, cols = input.shape

    # Índices GLOBALES del hilo, o sea, índices de la matriz completa, que hay que tener en cuenta
    # que no se salga de la propia matriz.
    x, y = cuda.grid(2)

    # Índices LOCALES del hilo, o sea, índices que usa el hilo para el bloque al cual pertenece
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # Cargamos los datos en memoria compartida, manejando los bordes, haciendo así la transformación
    # de arriba
    if 0 <= x < rows and 0 <= y < cols:
        shared_mem[tx + 1, ty + 1] = input[x, y]
    else:
        shared_mem[tx + 1, ty + 1] = 0.0

    # Cargar las celdas de los bordes
    if tx == 0 and x > 0:
        shared_mem[0, ty + 1] = input[x - 1, y] if (y < cols) else 0.0
    if tx == cuda.blockDim.x - 1 and x < rows - 1:
        shared_mem[tx + 2, ty + 1] = input[x + 1, y] if (y < cols) else 0.0
    if ty == 0 and y > 0:
        shared_mem[tx + 1, 0] = input[x, y - 1] if (x < rows) else 0.0
    if ty == cuda.blockDim.y - 1 and y < cols - 1:
        shared_mem[tx + 1, ty + 2] = input[x, y + 1] if (x < rows) else 0.0

    # Cargar las celdas de las esquinas
    if tx == 0 and ty == 0 and x > 0 and y > 0:
        shared_mem[0, 0] = input[x - 1, y - 1]
    if tx == cuda.blockDim.x - 1 and ty == 0 and x < rows - 1 and y > 0:
        shared_mem[tx + 2, 0] = input[x + 1, y - 1]
    if tx == 0 and ty == cuda.blockDim.y - 1 and x > 0 and y < cols - 1:
        shared_mem[0, ty + 2] = input[x - 1, y + 1]
    if tx == cuda.blockDim.x - 1 and ty == cuda.blockDim.y - 1 and x < rows - 1 and y < cols - 1:
        shared_mem[tx + 2, ty + 2] = input[x + 1, y + 1]

    # Esperamos a que todos los hilos terminen de cargar los datos en la memoria compartida
    cuda.syncthreads()

    # Calcular la media 3x3 si estamos dentro del rango
    if x < rows and y < cols:
        total_sum = 0.0
        if y != 0 and y != cols-1 and x != 0 and x != rows-1:
            # Sumar los 8 vecinos y la celda central
            total_sum = (shared_mem[tx, ty] + shared_mem[tx, ty + 1] + shared_mem[tx, ty + 2] +
                        shared_mem[tx + 1, ty] + shared_mem[tx + 1, ty + 1] + shared_mem[tx + 1, ty + 2] +
                        shared_mem[tx + 2, ty] + shared_mem[tx + 2, ty + 1] + shared_mem[tx + 2, ty + 2])
        elif y == 0 and x == rows-1:
            total_sum = (shared_mem[tx, ty + 1]     +  shared_mem[tx, ty + 2] +
                         shared_mem[tx + 1, ty + 1] +  shared_mem[tx + 1, ty + 2])
        elif y == cols-1 and x == rows-1:
            total_sum = (shared_mem[tx, ty]    +  shared_mem[tx, ty + 1] +
                        shared_mem[tx + 1, ty] +  shared_mem[tx + 1, ty + 1])
        elif y == 0:
            total_sum = (shared_mem[tx, ty + 1] + shared_mem[tx, ty + 2] +
                        shared_mem[tx + 1, ty + 1] + shared_mem[tx + 1, ty + 2] +
                        shared_mem[tx + 2, ty + 1] + shared_mem[tx + 2, ty + 2])
        elif y == cols-1:
            total_sum = (shared_mem[tx, ty] + shared_mem[tx, ty + 1] +
                        shared_mem[tx + 1, ty] + shared_mem[tx + 1, ty + 1] +
                        shared_mem[tx + 2, ty] + shared_mem[tx + 2, ty + 1])
        elif x == 0:
            total_sum = (shared_mem[tx + 1, ty] + shared_mem[tx + 1, ty + 1] + shared_mem[tx + 1, ty + 2] +
                        shared_mem[tx + 2, ty] + shared_mem[tx + 2, ty + 1] + shared_mem[tx + 2, ty + 2])
        elif x == rows-1:
            total_sum = (shared_mem[tx, ty] + shared_mem[tx, ty + 1] + shared_mem[tx, ty + 2] +
                        shared_mem[tx + 1, ty] + shared_mem[tx + 1, ty + 1] + shared_mem[tx + 1, ty + 2])

        output[x, y] = total_sum / 9.0
        aux[x+1,y+1] = shared_mem[tx+1, ty+1]



In [52]:
np.random.seed(0)

Ax = 50
Ay = 50
A = np.random.rand(Ax, Ay)
B = np.zeros_like(A)

#TODO: calculo del tiempo en cpu

Mean_3x3_sequential(A, B)
print("input\n", A)
print("output\n", B)

input
 [[0.25343217 0.88453917 0.11655505 ... 0.58155069 0.3772749  0.65740254]
 [0.54746975 0.53939572 0.66098    ... 0.38421246 0.60397874 0.98743678]
 [0.49014738 0.66282884 0.66250477 ... 0.3242048  0.09154355 0.27138173]
 ...
 [0.04962472 0.31829174 0.66001656 ... 0.53019427 0.87677016 0.35581574]
 [0.82315862 0.5766718  0.82800638 ... 0.83812446 0.10684913 0.36040019]
 [0.43248398 0.68363741 0.8608957  ... 0.8023918  0.60685719 0.13238978]]
output
 [[0.24720409 0.33359687 0.34379535 ... 0.35326124 0.39909512 0.29178811]
 [0.37531256 0.53531698 0.51184538 ... 0.4056532  0.47544291 0.33211314]
 [0.38543802 0.55499776 0.50469588 ... 0.43520547 0.51998738 0.36539528]
 ...
 [0.28819678 0.46406733 0.45136062 ... 0.52348112 0.51977772 0.31903569]
 [0.32042981 0.58142077 0.65604945 ... 0.57224097 0.51219919 0.27100913]
 [0.2795502  0.46720599 0.45870432 ... 0.3854164  0.31633473 0.13405514]]


In [53]:
B = np.zeros_like(A)
B_aux = np.zeros_like(np.random.rand(Ax+2, Ay+2))

A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
aux_device = cuda.to_device(B_aux)

threads_per_block = (16,16)

blocks_X = math.ceil(B_aux.shape[0] / threads_per_block[0])
blocks_Y = math.ceil(B_aux.shape[1] / threads_per_block[1])
blocks_total = (blocks_X, blocks_Y)

#TODO: calculo del tiempo en gpu

Mean_3x3_parallel[blocks_total, threads_per_block](A_device, B_device, aux_device)

B_par = B_device.copy_to_host()
aux = aux_device.copy_to_host()

print("input\n", A)
print("output\n", B_par)

#TODO: eliminación de la varaible auxiliar

print("memory\n", aux)

input
 [[0.25343217 0.88453917 0.11655505 ... 0.58155069 0.3772749  0.65740254]
 [0.54746975 0.53939572 0.66098    ... 0.38421246 0.60397874 0.98743678]
 [0.49014738 0.66282884 0.66250477 ... 0.3242048  0.09154355 0.27138173]
 ...
 [0.04962472 0.31829174 0.66001656 ... 0.53019427 0.87677016 0.35581574]
 [0.82315862 0.5766718  0.82800638 ... 0.83812446 0.10684913 0.36040019]
 [0.43248398 0.68363741 0.8608957  ... 0.8023918  0.60685719 0.13238978]]
output
 [[0.24720409 0.33359689 0.34379533 ... 0.35326126 0.39909511 0.2917881 ]
 [0.37531257 0.535317   0.51184538 ... 0.40565321 0.47544294 0.33211316]
 [0.38543802 0.55499776 0.50469589 ... 0.43520549 0.51998742 0.36539528]
 ...
 [0.2881968  0.46406735 0.4513606  ... 0.5234811  0.51977772 0.31903566]
 [0.3204298  0.58142074 0.65604941 ... 0.57224099 0.51219919 0.27100915]
 [0.27955018 0.46720595 0.45870431 ... 0.3854164  0.31633475 0.13405514]]
memory
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.2

In [ ]:
#TODO: calculo del speedup

# Optional assignment #2: Reduction approaches (3 points)

In [2]:
def Reduce_sequential(input, output):
    acum = 0
    for i in range(input.shape[0]):
        acum += input[i]
    output[0] = acum


N = 40
np.random.seed(0)
A = np.random.rand(N)
output = np.zeros(1)

#TODO: calculo del tiempo en cpu

Reduce_sequential(A, output)

print("Input\n", A)
print("Output\n", output[0])

Input
 [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203 ]
Output
 22.988006826188062


## Reduction #1: Interleaved addressing with divergent branching

In [57]:
################################ Explicaciones pal iñigo ################################

@cuda.jit
def reduce_interleaved_divergent(input, output):
    # Se piden crear dos listas en memoria compartida
    sSrc = cuda.shared.array(16, dtype=float32)
    sDst = cuda.shared.array(16, dtype=float32)

    # Id del hilo en el bloque
    tid = cuda.threadIdx.x
    # Id del bloque del programa
    bid = cuda.blockIdx.x
    # Nº de hilos que tiene el bloque
    bx = cuda.blockDim.x

    # Copiamos los datos del input a la memoria compartida
    if tid + bid * bx < input.size:
        # Este es el metodo por excelencia para guardar los datos en una lista
        # en el que no se solapa la posición con ningun otro hilo, en caso de
        # no comprenderlo dimelo
        sSrc[tid] = input[tid + bid * bx]
    else:
        # En caso de que el hilo no tenga posición del array no pone valor
        sSrc[tid] = 0
    cuda.syncthreads()

    # El stride es la división del hilo que se va multiplicando por 2 para que
    # se vaya minimizando el uso de hilos, haciendo en cada iteración:
    # nºhilos/2 -> nºhilos/4 ...
    stride = 2
    while stride <= bx:
        if tid % stride == 0:
            if tid + stride // 2 < bx:
                # sSrc[tid + stride // 2] lo dan en el enunciado
                sDst[tid] = sSrc[tid] + sSrc[tid + stride // 2]
        cuda.syncthreads()

        # Se copia la lista resultado a la lista de elementos para la siguiente
        # iteración
        sSrc[tid] = sDst[tid]
        cuda.syncthreads()

        # Como se indica arriba, se multiplica stride * 2
        stride *= 2

    # El primer hilo de todos tiene el privilegio de guardar el resultado
    if tid == 0:
        cuda.atomic.add(output, 0, sDst[0])

In [58]:
output = np.zeros(1)

input_device = cuda.to_device(A)
output_device = cuda.to_device(output)

threads_per_block = 16
blocks_per_grid = math.ceil(N + threads_per_block - 1 / threads_per_block)

#TODO: calculo del tiempo en gpu

reduce_interleaved_divergent[blocks_per_grid, threads_per_block](input_device, output_device)

output = output_device.copy_to_host()

print("Input\n", A)
print("Output\n", output[0])

Input
 [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203 ]
Output
 22.988006591796875


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 56 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
#TODO: calculo del speedup

## Reduction #2: Interleaved addressing with no divergent branching

In [3]:
@cuda.jit
def reduce_interleaved_no_divergent(input, output):
    # Allocate shared memory based on the number of threads per block (bx)
    sSrc = cuda.shared.array(shape=16, dtype=float32)  # Dynamically set the size in launch configuration
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bx = cuda.blockDim.x

    # Load data into shared memory with bounds check
    idx = tid + bid * bx
    if idx < input.size:
        sSrc[tid] = input[idx]
    else:
        sSrc[tid] = 0.0  # Handle out-of-bounds threads
    cuda.syncthreads()  # Ensure all threads have loaded their data

    # Interleaved reduction without branching
    stride = 1
    while stride < bx:
        index = tid + stride
        if index < bx:  # Ensure we do not access out-of-bounds in shared memory
            sSrc[tid] += sSrc[index]
        cuda.syncthreads()  # Synchronize all threads before the next iteration
        stride *= 2

    # Only the first thread in each block writes the result to the output
    if tid == 0:
        cuda.atomic.add(output, 0, sSrc[0])

In [4]:
output = np.zeros(1)

input_device = cuda.to_device(A)
output_device = cuda.to_device(output)

threads_per_block = 16
blocks_per_grid = math.ceil(N + threads_per_block - 1 / threads_per_block)

#TODO: calculo del tiempo en gpu

reduce_interleaved_no_divergent[blocks_per_grid, threads_per_block](input_device, output_device)

output = output_device.copy_to_host()

print("Input\n", A)
print("Output\n", output[0])

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 56 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Input
 [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203 ]
Output
 22.988006591796875


In [ ]:
#TODO: calculo del speedup

## Reduction #3: Sequential addressing

In [5]:
@cuda.jit
def reduce_sequential_addressing(input, output):
    sSrc = cuda.shared.array(16, dtype=float32)

    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bx = cuda.blockDim.x

    if tid + bid * bx < input.size:
        sSrc[tid] = input[tid + bid * bx]
    else:
        sSrc[tid] = 0
    cuda.syncthreads()

    stride = 1
    while stride < bx:
        index = 2 * stride * tid
        if index + stride < bx:
            sSrc[index] += sSrc[index + stride]
        cuda.syncthreads()

        stride *= 2

    if tid == 0:
        cuda.atomic.add(output, 0, sSrc[0])

In [6]:
output = np.zeros(1)

input_device = cuda.to_device(A)
output_device = cuda.to_device(output)

threads_per_block = 16
blocks_per_grid = math.ceil(N + threads_per_block - 1 / threads_per_block)

#TODO: calculo del tiempo en gpu

reduce_sequential_addressing[blocks_per_grid, threads_per_block](input_device, output_device)

output = output_device.copy_to_host()

print("Input\n", A)
print("Output\n", output[0])

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 56 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Input
 [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203 ]
Output
 22.988006591796875


In [ ]:
#TODO: calculo del speedup

## Reduction #4: Atomic addition

In [ ]:
@cuda.jit
def reduce_atomic_addition(input, output):

    partial_sum = cuda.shared.array(16, dtype=float32)

    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bx = cuda.blockDim.x

    if tid == 0:
        partial_sum[0] = 0.0
    cuda.syncthreads()

    if tid + bid * bx < input.size:
        cuda.atomic.add(partial_sum, 0, input[tid + bid * bx])
    cuda.syncthreads()

    if tid == 0:
        cuda.atomic.add(output, 0, partial_sum[0])

In [ ]:
output = np.zeros(1)

input_device = cuda.to_device(A)
output_device = cuda.to_device(output)

threads_per_block = 16
blocks_per_grid = math.ceil(N + threads_per_block - 1 / threads_per_block)

#TODO: calculo del tiempo en gpu

reduce_atomic_addition[blocks_per_grid, threads_per_block](input_device, output_device)

output = output_device.copy_to_host()

print("Input\n", A)
print("Output\n", output[0])

Input
 [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203 ]
Output
 22.988006114959717


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 56 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
#TODO: calculo del speedup